<a href="https://colab.research.google.com/github/rojinadeuja/NLP-Model-Implementations/blob/master/Word2vec_using_Keras_IMDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Import Modules

In [2]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords

from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten
from keras.layers import GlobalMaxPooling1D
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer

## Load Datatset

In [3]:
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/IMDB.csv')
df.replace(['positive', 'negative'], [1, 0], inplace=True)
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


## Data Pre-processing

In [4]:
def preprocess(s):
    '''Function for data pre-processing'''
    # Removing html tags
    TAG_RE = re.compile(r'<[^>]+>')
    sentence = TAG_RE.sub('', s)
    
    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)

    return sentence

## Create X and y matrices

In [5]:
X = []
sentences = list(df['review'])
for sentence in sentences:
    X.append(preprocess(sentence))

y = df['sentiment']

## Split dataset into train and test

In [6]:
# Create train-test splits
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
print(len(X_train), len(X_test), len(y_train), len(y_test))

40000 10000 40000 10000


In [7]:
# Convert into numpy arrays for processing with tensorflo
y_train = np.array(y_train)
y_test = np.array(y_test)
print(type(X_train), type(X_test), type(y_train), type(y_test))

<class 'list'> <class 'list'> <class 'numpy.ndarray'> <class 'numpy.ndarray'>


## Tokenize the data

In [8]:
# Tokenize the text
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [9]:
# Adding 1 because of reserved 0 index
vocab_size = len(tokenizer.word_index) + 1

maxlen = 100

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

## Create Embeddings

In [10]:
# import gensim
# path = '/content/drive/My Drive/Colab Notebooks/enwiki_20180420_100d.txt.bz2'
# embeddings = gensim.models.KeyedVectors.load_word2vec_format(path, binary = False)

from gensim.models import Word2Vec
model = Word2Vec.load('/content/drive/My Drive/Colab Notebooks/w2v_model_IMDB_100')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [11]:
embedding_matrix = np.zeros((len(model.wv.vocab), 100)) # vector_dim =100
for i in range(len(model.wv.vocab)):
    embedding_vector = model.wv[model.wv.index2word[i]]
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

## Classification using a simple Neural Network

In [13]:
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten
from keras.layers import GlobalMaxPooling1D
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
# Text Classification with a Simple Neural Network
vocab_size = len(model.wv.vocab)
model = Sequential()
embedding_layer = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=maxlen , trainable=False)
model.add(embedding_layer)

model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

In [14]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 100)          3364700   
_________________________________________________________________
flatten (Flatten)            (None, 10000)             0         
_________________________________________________________________
dense (Dense)                (None, 1)                 10001     
Total params: 3,374,701
Trainable params: 10,001
Non-trainable params: 3,364,700
_________________________________________________________________
None


## Train the model

In [17]:
# Train the model
history = model.fit(X_train, y_train, batch_size=128, epochs=10, verbose=1, validation_split=0.2)

Epoch 1/10
250/250 [==============================] - 1s 4ms/step - loss: 0.5069 - acc: 0.7605 - val_loss: 0.6734 - val_acc: 0.6292
Epoch 2/10
250/250 [==============================] - 1s 4ms/step - loss: 0.4981 - acc: 0.7667 - val_loss: 0.6750 - val_acc: 0.6300
Epoch 3/10
250/250 [==============================] - 1s 4ms/step - loss: 0.4929 - acc: 0.7660 - val_loss: 0.6864 - val_acc: 0.6279
Epoch 4/10
250/250 [==============================] - 1s 4ms/step - loss: 0.4861 - acc: 0.7678 - val_loss: 0.6907 - val_acc: 0.6289
Epoch 5/10
250/250 [==============================] - 1s 4ms/step - loss: 0.4808 - acc: 0.7725 - val_loss: 0.6992 - val_acc: 0.6260
Epoch 6/10
250/250 [==============================] - 1s 4ms/step - loss: 0.4771 - acc: 0.7747 - val_loss: 0.7055 - val_acc: 0.6300
Epoch 7/10
250/250 [==============================] - 1s 4ms/step - loss: 0.4701 - acc: 0.7806 - val_loss: 0.7196 - val_acc: 0.6258
Epoch 8/10
250/250 [==============================] - 1s 4ms/step - loss: 0.

## Evaluate model on test data

In [18]:
# Evaluate the model
score = model.evaluate(X_test, y_test, verbose=1)
print("\nTest Accuracy:", score[1])

313/313 [==============================] - 1s 2ms/step - loss: 0.7175 - acc: 0.6295

Test Accuracy: 0.6294999718666077
